<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/Random_forest_Stock_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector
 

In [2]:


df = pd.read_csv("HPQ.csv")
df.dropna(axis=0,inplace=True)

df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative
750,24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative
751,25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative
752,28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative
753,29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative


In [3]:
df['FutureTrend'].value_counts()

Negative    412
Positive    328
Neutral      14
Name: FutureTrend, dtype: int64

In [4]:
# Setting Index as Date
appl_df = df.set_index("Date")
appl_df.tail()
#drop na
appl_df.dropna(axis=0,inplace=True)

In [5]:
def make_bins(df):
   label_names = ["Ef_1_v_low","Ef_1_low","Ef_1_v_med","Ef1_high", "Ef1_very_high" ]
   cut_points =[0. , 0.2, 0.4, 0.6, 0.8, 1. ]
   df["EF_1_val"] = pd.cut(df["EF_1"], cut_points, labels=label_names)
   dummies = pd.get_dummies(df["EF_1_val"],drop_first=True)
   df = pd.concat([df, dummies], axis=1)
   label_names = ["setinent_neutral","sentiment_good","sentiment_bad"]
   cut_points =[0, 0.29,0.58 ,0.87] 
   df["sentiments"]=pd.cut(df['Twitter Sentiments'], cut_points, labels=label_names)
   dummies1 = pd.get_dummies(df["sentiments"],drop_first=True)
   df = pd.concat([df, dummies1], axis=1)
   df.drop(["EF_1_val","sentiments"],axis=1,inplace=True)
   return df

def calculate_ema(prices, days, smoothing=2):
      ema = [sum(prices[:days]) / days]
      for price in prices[days:]:
          ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
      return ema

In [6]:
appl_df.head()

,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
Date,,,,,,,,,,,,
02/07/2018,22.540001,22.860001,22.379999,22.860001,5855900,1.000000,1.419697,Positive,Negative,Positive,0,Positive
03/07/2018,22.920000,22.930000,22.549999,22.590000,3619200,0.105265,-1.439790,Neutral,Neutral,Negative,0,Negative
05/07/2018,22.820000,22.990000,22.450001,22.750000,7158600,0.555555,-0.306747,Negative,Positive,Negative,0,Negative
06/07/2018,22.780001,23.219999,22.600000,23.010000,5375100,0.661291,1.009656,Negative,Negative,Positive,0,Negative
09/07/2018,23.139999,23.440001,23.010000,23.370001,7057100,0.837210,0.993956,Negative,Positive,Positive,0,Negative


In [7]:
target=appl_df['FutureTrend']
appl_df=appl_df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
appl_df = make_bins(df)
appl_df.head(2)
appl_df=appl_df.drop(["Twitter Sentiments"],axis =1)
appl_df['diff']=appl_df['Close']-appl_df['Open']
appl_df['10day']=appl_df['Close'].rolling(10).mean()
appl_df['10day'].fillna(appl_df['10day'][9],inplace=True)
  
ema = calculate_ema(appl_df['Close'], 10)
appl_df=appl_df[9:]
target=target[9:]
appl_df['ema']=ema
appl_df['signal']=appl_df['ema']>appl_df['Close']

del (appl_df['Open'])
del (appl_df['Close'])
del (appl_df['High'])
del (appl_df['EF_2'])

# appl_df['normVol'] = appl_df['Volume'] / appl_df['Volume'].ewm(5).mean()   

appl_df.head()

,Date,Low,Volume,EF_1,Oil FutureTrend,Gold FutureTrend,Trend,FutureTrend,Ef_1_low,Ef_1_v_med,Ef1_high,Ef1_very_high,sentiment_good,sentiment_bad,diff,10day,ema,signal
9,16/07/2018,23.510000,3791500,0.260868,Negative,Positive,Negative,Negative,1,0,0,0,0,0,-0.120001,23.232,23.232000,False
10,17/07/2018,23.379999,4094700,0.617650,Negative,Negative,Positive,Positive,0,0,1,0,0,0,0.210001,23.305,23.297091,False
11,18/07/2018,23.320000,6467400,0.583335,Negative,Positive,Negative,Positive,0,1,0,0,0,0,-0.070000,23.399,23.339438,False
12,19/07/2018,23.170000,6056000,0.055557,Negative,Negative,Negative,Positive,0,0,0,0,0,0,-0.209999,23.443,23.312268,True
13,20/07/2018,22.889999,6919000,0.161293,Positive,Positive,Negative,Negative,0,0,0,0,0,0,-0.250000,23.436,23.244583,True


In [8]:
new_df=appl_df
new_df.set_index('Date',inplace=True)
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('minmax-scaler', numerical_preprocessor, numerical_columns)])



['Oil FutureTrend', 'Gold FutureTrend', 'Trend', 'FutureTrend']


In [9]:

model_rf1 =make_pipeline(preprocessor,RandomForestClassifier(n_estimators=30,
                                      max_features=2,
                                      max_depth=3,
                                      random_state=0))
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.1, random_state=42)
  # Scoring the model
model_rf1.fit(X_train,y_train)
print("Accuracy score (training): {0:.3f}".format(
        model_rf1.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_rf1.score(
            X_test, 
            y_test)))


Accuracy score (training): 0.982
Accuracy score (validation): 0.987


In [10]:
df_t=preprocessor.fit_transform(new_df)
df_t

array([[ 1.        ,  0.        ,  0.        , ...,  0.33469816,
         0.33669055, -0.79384944],
       [ 1.        ,  0.        ,  0.        , ...,  0.35108704,
         0.35137542, -0.79384944],
       [ 1.        ,  0.        ,  0.        , ...,  0.37219054,
         0.36092918, -0.79384944],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  1.67566784,
         1.73674214, -0.79384944],
       [ 1.        ,  0.        ,  0.        , ...,  1.68509706,
         1.75201146, -0.79384944],
       [ 1.        ,  0.        ,  0.        , ...,  1.69744483,
         1.77804086, -0.79384944]])

train_test splt

In [11]:


from sklearn.pipeline import make_pipeline

model_rf =RandomForestClassifier(n_estimators=30,
                                      max_features=2,
                                      max_depth=3,
                                      random_state=0)

In [12]:
X_train,X_test, y_train, y_test = train_test_split(df_t, target, test_size=0.1, random_state=42)

In [13]:
  # Scoring the model
model_rf.fit(X_train,y_train)
print("Accuracy score (training): {0:.3f}".format(
        model_rf.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_rf.score(
            X_test, 
            y_test)))


Accuracy score (training): 0.982
Accuracy score (validation): 0.987


Hyper 

In [14]:
param_grid = {
'n_estimators': [50, 100, 200, 300],
'min_samples_leaf': [1, 5, 10],
'max_depth': [2, 4, 6, 8, 10],
'max_features': ['auto', 'sqrt'],
'bootstrap': [True, False]}

In [15]:
dff=preprocessor.fit_transform(new_df)


X_train, X_test, y_train, y_test = train_test_split(dff, target, train_size= 0.2,shuffle=False)
from imblearn.over_sampling import SMOTE

# pd.DataFrame(oe_df)
rf=RandomForestClassifier()
model_gridsearch =   GridSearchCV(
estimator=rf,
param_grid=param_grid,
scoring='accuracy',

cv=5,
refit=True,
return_train_score=True)

Grid search

In [16]:
model_gridsearch.fit(X_train, y_train)
# Print the time spend and number of models ran
print("GridSearchCV took %.2f seconds for %d candidate parameter settings." ,len(model_gridsearch.cv_results_['params']))

GridSearchCV took %.2f seconds for %d candidate parameter settings. 240


In [17]:
y_pred_grid = model_gridsearch.predict(X_test)
accuracy_grid = accuracy_score(y_test, y_pred_grid)


In [18]:
print(model_gridsearch.best_params_)
print(model_gridsearch.best_score_)

{'bootstrap': True, 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 50}
1.0


In [19]:
print (accuracy_grid)
print(y_pred_grid)

1.0
[0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1
 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0
 0 0 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 2 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1
 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 1 2 1 0
 1 0 1 0 0 0 0 1 2 1 1 0 1 0 1 0 0 0 0 1 1 0 0 2 2 0 0 1 0 1 0 0 0 0 1 0 0
 1 2 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 1 1 1 0 2 0 0 1 1 0 0 1 0
 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1
 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0
 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0
 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 2 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0
 0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0
 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1
 1 1 0 0 1 1 1 0 0 1 

In [20]:
 np.array(np.unique(target, return_counts=True)).T

array([[  0, 406],
       [  1, 325],
       [  2,  14]])

In [21]:
model_rf = RandomForestClassifier(bootstrap= True, max_depth= 10, max_features= 'auto', min_samples_leaf= 1, n_estimators= 200)
model_rf.fit(X_train,y_train)
  # Scoring the model
print("Accuracy score (training): {0:.3f}".format(
        model_rf.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_rf.score(
            X_test, 
            y_test)))


Accuracy score (training): 1.000
Accuracy score (validation): 1.000


In [22]:
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend,EF_1_val
749,23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative,Ef_1_v_med
750,24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative,Ef1_very_high
751,25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative,Ef1_high
752,28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative,Ef_1_v_med
753,29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative,Ef1_high


In [ ]:
# new_df=appl_df
# new_df

# numerical_columns_selector = selector(dtype_exclude=object)
# categorical_columns_selector = selector(dtype_include=object)
# numerical_columns = numerical_columns_selector(new_df)
# categorical_columns = categorical_columns_selector(new_df)
# print(categorical_columns)
# preprocessor = ColumnTransformer([
#     ('one-hot-encoder', categorical_preprocessor, categorical_columns),
#     ('standard-scaler', numerical_preprocessor, numerical_columns)])

# model = make_pipeline(preprocessor,  RandomForestRegressor(n_estimators=30,
#                                       max_features=2,
#                                       max_depth=3,
#                                       random_state=2))
# model.fit(X_train,y_train)

In [ ]:
#   # Scoring the model
# print("Accuracy score (training): {0:.3f}".format(
#         model.score(
#             X_train, 
#             y_train)))
# print("Accuracy score (validation): {0:.3f}".format(
#         model.score(
#             X_test, 
#             y_test)))


In [ ]:
# parameters = {
#     "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#     "max_depth":[3,5,8],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
#     "n_estimators":[10,20,40,50]
#     }
# from sklearn.compose import make_column_selector as selector
# numerical_columns_selector = selector(dtype_exclude=object)
# categorical_columns_selector = selector(dtype_include=object)
# numerical_columns = numerical_columns_selector(new_df)
# categorical_columns = categorical_columns_selector(new_df)
       
# preprocessor = ColumnTransformer([
#     ('one-hot-encoder', categorical_preprocessor, categorical_columns),
#     ('standard-scaler', numerical_preprocessor, numerical_columns)])


# new_df=preprocessor.fit_transform(new_df)
# model_gbm=GradientBoostingClassifier()
# clf = GridSearchCV(model_gbm, parameters,refit=False,cv=2, n_jobs=-1)

# clf.fit(X_train, y_train)
# print("Accuracy score (training): {0:.3f}".format(
#         clf.score(
#             X_train, 
#             y_train)))
# print("Accuracy score (validation): {0:.3f}".format(
#         clf.score(
#             X_test, 
#             y_test)))

GBM

In [ ]:
# predictions=model.predict(X_test)
# stocks = pd.DataFrame({
#     "Real": y_test.ravel(),
#     "Predicted": predictions
# }, index = df.index[-len(y_test): ]) 
# stocks.tail(100)


In [ ]:
# # Generating the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1","Actual 2"],
#     columns=["Predicted 0", "Predicted 1", "Predicted 2"]
# )

# # Displaying results
# #display(cm_df)

In [ ]:
# def compute_accuracy(y_true, y_pred):
#     correct_predictions = 0
#     # iterate over each label and check
#     for true, predicted in zip(y_true, y_pred):
#         if true == predicted:
#             correct_predictions += 1
#     # compute the accuracy
#     accuracy = correct_predictions/len(y_true)
#     return accuracy

In [ ]:
# # Choosing learning rate
# from sklearn.ensemble import RandomForestClassifier
# learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 2, 3, 4]
# for learning_rate in learning_rates:
#     model = RandomForestRegressor(n_estimators=20,
#                                       max_features=2,
#                                       max_depth=3,
#                                       random_state=0)
#     predictions=model.predict(X_test)
#     np.where(predictions > 0.5, 1, 0)
#     compute_accuracy(y_test, predictions):
  
#     model.fit(X_train,y_train)
#     print("Learning rate: ", learning_rate)
    
#     # Scoring the model
#     print("Accuracy score (training): {0:.3f}".format(
#         model.score(
#             X_train, 
#             y_train)))
#     print("Accuracy score (validation): {0:.3f}".format(
#         model.score(
#             X_test,
#             y_test)))
#     print()